In [1]:
!pip install spacy_transformers
!pip install -U spacy

  Using cached spacy_transformers-1.3.8-cp312-cp312-win_amd64.whl.metadata (7.2 kB)
  Using cached spacy-3.8.4-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached thinc-8.3.4-cp312-cp312-win_amd64.whl.metadata (15 kB)
Using cached spacy_transformers-1.3.8-cp312-cp312-win_amd64.whl (343 kB)
Using cached spacy-3.8.4-cp312-cp312-win_amd64.whl (11.8 MB)
Using cached thinc-8.3.4-cp312-cp312-win_amd64.whl (1.5 MB)


In [ ]:
def json_parser(data):
  parsed_data = []
  for item in data:
    text = item["text"]
    labels = []
    for entity in item["label"]:
      label = entity["labels"][0].capitalize()
      start = entity["start"]
      end = entity["end"]
      labels.append((start, end, label))
    parsed_data.append((text, labels))
  return parsed_data

In [2]:
def pubtator_extractor(data):
  """Converts data from pubtator to Spacy's JSON Format"""
  parsed_data = []
  for line in data:
    line = line.strip()
    if line == "":
       if parsed_entity:
        parsed_data.append(tuple(parsed_entity))
    if "|t|" in line:
      parsed_entity = []
      current_title = line.split("|t|")[1] + " "
    elif "|a|" in line:
      parsed_entity.append(current_title + line.split("|a|")[1])
    elif "Disease" in line or "Chemical" in line:
      if len(parsed_entity) == 1:
        parsed_entity.append([])
      start = int(line.split("\t")[1])
      end = int(line.split("\t")[2])
      label = line.split("\t")[4]
      parsed_entity[1].append((start, end, label))
  return parsed_data

In [3]:
with open("data/CDR_DevelopmentSet.PubTator.txt", mode="r") as f:
  dev_data = pubtator_extractor(f.readlines())
  f.close()

In [4]:
# Let's import train_data json file and parse it to spacy format
import json
with open("data/initial_annotated_train.json", mode="r") as f:
  train_data = json_parser(json.load(f))
  f.close()

In [5]:
# Let's Verify the data struct 
print(train_data[0])
print(dev_data[0])

('Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release

In [6]:
import spacy
from spacy.tokens import DocBin

def db_creator(data):
  db = DocBin()
  nlp = spacy.blank("en")

  for text, annotations in data:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
      span = doc.char_span(start, end, label=label)
      if span is None:
        continue
      else:   
        ents.append(span)
      doc.ents = ents
    db.add(doc)
  return db

In [7]:
train = db_creator(train_data)
train.to_disk("train.spacy")

dev = db_creator(dev_data)
dev.to_disk("dev.spacy")

In [9]:
!python -m spacy init fill-config ./spacy/base_config.cfg ./spacy/config.cfg

[+] Auto-filled config with all values
[+] Saved config
spacy\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy train ./spacy/config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

^C
